In [ ]:
# Cell 0: Hardware Check
import torch

def check_hardware():
    if torch.cuda.is_available():
        vram = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU Available: {torch.cuda.get_device_name(0)} ({vram:.1f} GB VRAM)")
    else:
        print("⚠️ GPU Not Available - Logic Verification Mode Only")

check_hardware()

# Aura-Med: Pediatric Respiratory Triage

**Problem:** Pneumonia is the #1 infectious killer of children worldwide (700,000 deaths/year).
**Solution:** Fast, offline, audible-to-actionable triage using HeAR and MedGemma.

In [ ]:
# Cell 1: Setup & Dependencies
!pip install -q torch transformers librosa pydantic
print("✅ Dependencies installed.")

In [ ]:
# Cell 2: Configuration
import os

# Resource Constraints
MAX_RAM_GB = 4.0
MAX_INFERENCE_TIME_SEC = 10.0

# Model Paths (Mock or Real)
HEAR_MODEL_PATH = "google/hear-encoder"
MEDGEMMA_MODEL_PATH = "google/medgemma-2b"

# Audio Settings
SAMPLE_RATE = 16000
MAX_AUDIO_DURATION_SEC = 10

# Demo Settings
IS_DEMO_MODE = True

In [ ]:
# Cell 3: Core Components & Data Structures
import librosa
import numpy as np
import torch
import os
from enum import Enum
from dataclasses import dataclass
from typing import Optional, Dict
from pydantic import BaseModel, Field
class TriageStatus(str, Enum):
    GREEN = "GREEN"
    YELLOW = "YELLOW"
    RED = "RED"
    INCONCLUSIVE = "INCONCLUSIVE"
class PatientVitals(BaseModel):
    age_months: int = Field(..., ge=0, description="Patient age in months")
    respiratory_rate: int = Field(..., ge=0, description="Breaths per minute")
    danger_signs: bool = Field(False, description="Presence of any general danger signs")
@dataclass
class TriageResult:
    status: TriageStatus
    confidence: float
    reasoning: str
    usage_stats: Optional[Dict[str, float]] = None
    action_recommendation: Optional[str] = None
class DangerSignException(Exception):
    """Raised when danger signs are detected, triggering immediate override."""
    pass
class LowQualityError(Exception):
    """Raised when audio quality/duration is insufficient."""
    pass
class LowConfidenceError(Exception):
    """Raised when model confidence is below threshold."""
    pass
class EdgeConstraintViolation(Exception):
    """Raised when resource usage exceeds edge limits."""
    pass
def load_audio(path: str, sr: int = 16000):
    """Load audio and resample to target rate."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Audio file not found: {path}")
    waveform, sample_rate = librosa.load(path, sr=sr, mono=True)
    return waveform, sample_rate
def normalize_duration(audio, target_length: float = 10.0, sr: int = 16000):
    """Truncate or zero-pad audio to target duration."""
    target_samples = int(target_length * sr)
    current_samples = len(audio)
    if current_samples > target_samples:
        return audio[:target_samples]
    elif current_samples < target_samples:
        return np.concatenate((audio, np.zeros(target_samples - current_samples)))
    return audio
class HeAREncoder:
    """Implementation of HeAR Encoder pipeline (preprocessing + mock inference)."""
    def __init__(self):
        print(f"Loading HeAR Encoder from {HEAR_MODEL_PATH}...")
    def encode(self, audio_path: str) -> torch.Tensor:
        """Extract embeddings after preprocessing audio."""
        waveform, sr = load_audio(audio_path, sr=SAMPLE_RATE)
        if len(waveform) < sr:
            raise LowQualityError("Audio recording is too short (min 1s required)")
        
        normalized_waveform = normalize_duration(waveform, target_length=MAX_AUDIO_DURATION_SEC, sr=sr)
        return torch.randn(1, 1024)


class MedGemmaReasoning:
    """
    Mock implementation of MedGemma Reasoning Engine.
    """
    def __init__(self):
        """Initialize the reasoning engine."""
        print(f"Loading MedGemma from {MEDGEMMA_MODEL_PATH}...")
        pass

    def generate(self, embedding, vitals: PatientVitals) -> TriageResult:
        """
        Generate triage result based on embedding and vitals.
        
        Args:
            embedding: HeAR embedding
            vitals: Patient vital signs
            
        Returns:
            TriageResult: Structured triage output
        """
        # Mock logic
        return TriageResult(
            status=TriageStatus.YELLOW,
            confidence=0.85,
            reasoning="Mock reasoning: Crackles detected consistent with pneumonia.",
            usage_stats={"ram_gb": 0.5},
            action_recommendation="Administer oral Amoxicillin. Follow up in 48 hours."
        )

# Execution Test
print("✅ Code synced successfully.")
print("Initializing Mock Models...")
hear = HeAREncoder()
medgemma = MedGemmaReasoning()
print("✅ Mock Models Initialized")